# Imports

In [ ]:
# imports
from __future__ import print_function, division
%matplotlib notebook

from niftypet import nipet
from niftypet import nimpa

from os import path
import matplotlib.pyplot as plt
import ipywidgets as ipyw
from tqdm.auto import trange
from brainweb import volshow
from scipy.ndimage.filters import gaussian_filter

import logging
logging.basicConfig(level=logging.INFO)

print(nimpa.gpuinfo())

# get all the scanner parameters
mMRpars = nipet.get_mmrparams()

# Load & Process Raw Data

In [ ]:
folderin = "amyloidPET_FBP_TP0"

# automatically categorise the input data
#logging.getLogger().setLevel(logging.INFO)
datain = nipet.classify_input(folderin, mMRpars)

# output path
opth = path.join(datain['corepath'], 'niftyout')
# switch on verbose mode
#logging.getLogger().setLevel(logging.DEBUG)

datain

In [ ]:
# hardware mu-map (bed, head/neck coils)
muhdct = nipet.hdw_mumap(datain, [1,2,4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# MR-based human mu-map

# UTE-based object mu-map aligned (need UTE sequence or T1 for pseudo-CT)
#muodct = nipet.align_mumap(
#    datain,
#    scanner_params=mMRpars,
#    outpath=opth,
#    t0=0, t1=0, # when both times are 0, will use full data
#    itr=2,      # number of iterations used for recon to which registering MR/UTE
#    petopt='ac',# what PET image to use (ac-just attenuation corrected)
#    musrc='ute',# source of mu-map (ute/pct)
#    ute_name='UTE2', # which UTE to use (UTE1/2 shorter/faster)
#    verbose=True,
#)

#> the same as above without any faff though (no alignment)
muodct = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

In [ ]:
# create histogram
hst = nipet.mmrhist(datain, mMRpars)

## Visualisations

In [ ]:
try:  # needs HW maps
    volshow(muodct['im'] + muhdct['im'], cmaps=['bone'])
except:
    volshow(muodct['im'], cmaps=['bone'])

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
volshow([hst['psino'], hst['dsino']],
        titles=['prompt sinogram (%.3gM)' % (hst['psino'].sum() / 1e6),
               'delayed sinogram (%.3gM)' % (hst['dsino'].sum() / 1e6)],
        cmaps=['inferno'] * 2, xlabels=['', 'bins'], ylabels=['angles'] * 2);

# Reconstruction

## OSEM 14 subsets

In [ ]:
recon = nipet.mmrchain(
    datain, mMRpars,
    frames=['timings', [3000, 3600]],
    mu_h=muhdct,
    mu_o=muodct,
    itr=4,
    histo=hst,
    fwhm=0.0,
    outpath=opth,
    fcomment='niftypet-recon',
    store_img=True)

volshow(recon['im'][:, 100:-100, 100:-100], cmaps=['magma']);

## MLEM

In [ ]:
## Randoms

rndsino, _ = nipet.randoms(hst, mMRpars)
#volshow(rndsino, cmaps=['inferno'])
print("Randoms: %.3g%%" % (rndsino.sum() / hst['psino'].sum() * 100))

In [ ]:
## Scatter

# One OSEM iteration estimate (implicitly using voxel-driven scatter model)
eim = nipet.mmrchain(
    datain, mMRpars, mu_h=muhdct, mu_o=muodct, itr=1, outpath=opth)['im']
# Recalculate scatter
sctsino, sssr, amsk = nipet.vsm(
    datain, (muhdct['im'], muodct['im']), eim, hst, rndsino, mMRpars)
#volshow(sctsino, cmaps=['inferno'])
print("Scatter: %.3g%%" % (sctsino.sum() / hst['psino'].sum() * 100))

## Attenuation, Normalisation & Sensitivity

attsino = nipet.frwd_prj(muodct['im'] + muhdct['im'], mMRpars, attenuation=True)
nrmsino = nipet.mmrnorm.get_norm_sino(datain, mMRpars, hst)
ansino = attsino * nrmsino
#volshow(ansino, title='Attenuation & Normalisation Sinogram',
#        cmap='inferno', xlabel='bins', ylabel='angles')
sim = nipet.back_prj(ansino, mMRpars)
#volshow(sim, cmaps=['inferno'])

In [ ]:
SIG2FWHM = np.array([4.783228, 4.912767, 4.912767])  # convert Gaussian sigma/[voxel] to FWHM/[mm]
psfSigma = 2.5 / SIG2FWHM
recon_mlem = [np.ones_like(sim)]
msk = nipet.img.mmrimg.get_cylinder(mMRpars['Cnt'], rad=29., xo=0., yo=0., unival=1, gpu_dim=False) <= 0.9
sim_inv = 1 / gaussian_filter(sim, psfSigma)
sim_inv[msk] = 0
rndsctsino = rndsino + sctsino
for k in trange(4 * 14, desc="MLEM"):
    fprj = nipet.frwd_prj(gaussian_filter(recon_mlem[-1], psfSigma), mMRpars) + rndsctsino
    recon_mlem.append(recon_mlem[-1] * sim_inv
                      * gaussian_filter(nipet.back_prj(hst['psino'] / fprj, mMRpars), psfSigma))

In [ ]:
# central slice across iterations
volshow(np.asanyarray(recon_mlem[1::5])[:, :, 100:-100, 100:-100], cmaps=['magma'] * len(recon_mlem[1::5]));